In [1]:
import time
import os

import pandas as pd
import requests
import csv

# Get Listing Status

In [247]:
# get listing status
api_key = open("key", "r").read()
url = 'https://www.alphavantage.co/query'
params = {
    "apikey": api_key,
    "function": "LISTING_STATUS",
    "outputsize": "full",
    "datatype": "csv"
}

response = requests.get(url, params=params)

response = response.content.decode('utf-8')
response = response.splitlines()
response = [line.split(',') for line in response]
# save to csv
with open('data/listing_status.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for row in response:
        writer.writerow(row)

In [248]:
df_symbols = pd.read_csv('data/listing_status.csv', delimiter=',')
# drop rows where name is NaN
df_symbols = df_symbols.dropna(subset=['name'])
# save to csv
df_symbols.to_csv('data/listing_status.csv', index=True)

# Methods to get Time Series data

In [38]:
error_msg = b'{\n    "Note": "Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 100 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency."\n}'
max_error = b'{\n    "Information": "Thank you for using Alpha Vantage! You have reached the 100 requests/day limit for your free API key. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits."\n}'

In [46]:
def get_response(api_key, function, symbol):
    params = {
        "apikey": api_key,
        "function": function,
        "symbol": symbol,
        "outputsize": "full",
        "datatype": "csv"
    }
    response = requests.get(url, params=params)
    if response.content == error_msg:
        print("Too many calls, sleep for 60 seconds")
        time.sleep(61)
        return get_response(api_key, function, symbol)
    if response.content == max_error:
        print("Max calls reached")
        return None
    return response


def modify_output(symbol, response):
    data = response.content.decode('utf-8')
    data = data.splitlines()
    # add symbol at the beginning of each line
    data = [symbol + "," + line for line in data]
    return data[1:]


def save_csv(path, data):
    with open(path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for row in data:
            writer.writerow(row.split(','))


def timestamps(api_key, function, symbol, path):
    response = get_response(api_key, function, symbol)
    if response is None:
        return None
    data = modify_output(symbol, response)
    save_csv(path, data)
    return "success"

# Get Time Series Daily

In [47]:
path = 'data/timestamps.csv'

if not os.path.isfile(path):
    # create file
    with open(path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["symbol", "timestamp", "open", "high", "low", "close", "volume"])
    print("File created")

In [48]:
calls = 100

In [49]:
# api_key = open("key", "r").read()
api_key = open("key_roman", "r").read()
url = 'https://www.alphavantage.co/query'

function = "TIME_SERIES_DAILY"

In [50]:
df_symbols = pd.read_csv('data/listing_status.csv', delimiter=',')
df_symbols = df_symbols[df_symbols['name'].str.contains("MSCI|FTSE")]
symbols = df_symbols["symbol"].unique()
print("Number of symbols: {}".format(len(symbols)))

Number of symbols: 260


In [51]:
df = pd.read_csv('data/timestamps.csv', delimiter=',')  # get all timestamps that are not a time format
done_symbols = df["symbol"].unique()
todo_symbols = [s for s in symbols if s not in done_symbols]
print("Number of symbols to do: {}".format(len(todo_symbols)))
print("Number of symbols done: {}".format(len(done_symbols)))
print("Percentage done: {:.2f}%".format(len(done_symbols) / len(symbols) * 100))

Number of symbols to do: 233
Number of symbols done: 27
Percentage done: 10.38%


In [52]:
for symbol in todo_symbols:
    if calls == 0:
        break
    data = timestamps(api_key, function, symbol, path)
    last_symbol = symbol
    if data == "success":
        print("Data for symbol {} saved successfully".format(symbol))
        calls -= 1
    else:
        print("Data for symbol {} not saved".format(symbol))
        break

Max calls reached
Data for symbol CUT not saved
